<a href="https://colab.research.google.com/github/emello23/Alura-Gemini/blob/main/Agente_nova_saida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pandas google-api-python-client google-generativeai


In [5]:
import pandas as pd
import time
from IPython.display import display, Markdown
from tabulate import tabulate
from urllib.parse import quote_plus
import google.generativeai as genai # SDK do Gemini
import json # Para parsear a resposta do Gemini
from google.colab import userdata # Para acessar a API key

# --- Configuração do Agente e API Key ---
GOOGLE_API_KEY = None
gemini_model_instance = None # Renomeado para clareza

try:
    # O nome padrão para secrets é sem o sufixo _, a menos que você tenha nomeado especificamente assim.
    # Vou usar 'GOOGLE_API_KEY' como padrão, conforme a prática comum.
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
        display(Markdown("<font color='red'>**Erro Crítico: GOOGLE_API_KEY não encontrada nos Secrets do Colab.** "
                         "Por favor, configure-a com o nome 'GOOGLE_API_KEY'.</font>"))
    else:
        genai.configure(api_key=GOOGLE_API_KEY)
        gemini_model_instance = genai.GenerativeModel(
            model_name='gemini-1.5-flash-latest', # Modelo eficiente e capaz
            # A opção response_mime_type pode ser usada com modelos mais recentes
            # e configurações de segurança apropriadas.
            # generation_config=genai.types.GenerationConfig(
            #     response_mime_type="application/json"
            # )
        )
        display(Markdown("✅ *SDK do Gemini e modelo (`gemini-1.5-flash-latest`) configurados com sucesso.*"))
except Exception as e:
    display(Markdown(f"<font color='red'>**Erro Crítico ao configurar o SDK do Gemini:** {e}. "
                     "Verifique sua API Key, permissões e se o nome do secret está correto ('GOOGLE_API_KEY'). "
                     "O agente não funcionará sem isso.</font>"))
    # O script poderia parar aqui ou continuar com funcionalidades limitadas se houvesse um fallback.
    # Para este agente, o Gemini é essencial.

# --- Funções Utilitárias ---

def carregar_dados_viagem(caminho_arquivo: str) -> pd.DataFrame | None:
    """
    Carrega os dados de viagem do arquivo CSV e valida as colunas necessárias.
    """
    colunas_requeridas = ['cidade', 'pais', 'data_chegada', 'data_partida', 'hospedagem']
    try:
        df = pd.read_csv(caminho_arquivo)

        # Validar se as colunas requeridas existem
        if not all(col in df.columns for col in colunas_requeridas):
            colunas_faltando = [col for col in colunas_requeridas if col not in df.columns]
            display(Markdown(f"<font color='red'>**Erro: O arquivo `{caminho_arquivo}` não contém as colunas requeridas.** "
                             f"Faltam as seguintes colunas: {', '.join(colunas_faltando)}.</font>"))
            return None

        display(Markdown(f"### Arquivo de Viagem (`{caminho_arquivo}`)"))
        display(df[colunas_requeridas]) # Exibe apenas as colunas relevantes
        return df
    except FileNotFoundError:
        display(Markdown(f"<font color='red'>**Erro: Arquivo `{caminho_arquivo}` não encontrado.** Verifique o nome e o local.</font>"))
        return None
    except Exception as e:
        display(Markdown(f"<font color='red'>**Erro ao carregar o arquivo CSV `{caminho_arquivo}`:** {e}</font>"))
        return None

def gerar_links_pesquisa_google(cidade: str, pais: str) -> tuple[str, str, str]:
    """Gera links de pesquisa úteis para Google Search e Google Maps."""
    query_atracoes = f"principais atrações turísticas em {cidade} {pais}"
    link_google_search = f"https://www.google.com/search?q={quote_plus(query_atracoes)}"
    query_maps_atracoes = f"atrações turísticas em {cidade}, {pais}"
    link_google_maps_atracoes = f"https://www.google.com/maps/search/{quote_plus(query_maps_atracoes)}"
    query_maps_cidade = f"{cidade}, {pais}"
    link_google_maps_cidade = f"https://www.google.com/maps/place/{quote_plus(query_maps_cidade)}"
    return link_google_search, link_google_maps_atracoes, link_google_maps_cidade

# --- Definição do Agente Pesquisador de Atrações ---

class AgentePesquisadorAtracoes:
    """
    Um agente responsável por pesquisar atrações turísticas usando o Gemini
    e fornecer links úteis para pesquisa manual.
    """
    def __init__(self, modelo_llm: genai.GenerativeModel):
        if modelo_llm is None:
            raise ValueError("O modelo LLM (Gemini) não pode ser None para este agente.")
        self.modelo = modelo_llm
        self.safety_settings = [ # Configurações de segurança para o Gemini
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        ]

    def _obter_sugestoes_gemini(self, cidade: str, pais: str, top_n: int = 7) -> list[dict]:
        """
        Ferramenta interna do agente para consultar o Gemini sobre atrações.
        """
        display(Markdown(f"🧠 *Agente consultando Gemini sobre atrações em: **{cidade}, {pais}** (Top {top_n})...*"))

        prompt = f"""
        Você é um assistente de planejamento de viagens altamente especializado e eficiente.
        Sua tarefa é identificar as {top_n} principais e mais recomendadas atrações turísticas para a cidade de {cidade}, localizada em {pais}.

        Critérios para sua seleção:
        1.  **Popularidade e Reconhecimento:** Atrações bem conhecidas e frequentemente visitadas.
        2.  **Qualidade das Avaliações:** Lugares geralmente com avaliações positivas (ex: acima de 4.0/5 estrelas).
        3.  **Relevância Cultural/Histórica:** Locais com significado importante.
        4.  **Diversidade de Experiências:** Inclua uma mistura de tipos, como museus, monumentos, parques, mercados, igrejas/catedrais, mirantes, etc., quando apropriado para a cidade.
        5.  **Singularidade:** Atrações que oferecem uma experiência única ou icônica da cidade/região.

        Formato da Resposta:
        Retorne **estritamente uma lista JSON**. Cada elemento da lista deve ser um dicionário representando uma atração,
        contendo EXATAMENTE as seguintes chaves:
        - "nome": (string) O nome oficial e completo da atração.
        - "tipo_principal": (string) A categoria principal da atração (ex: "Museu de Arte", "Catedral Gótica", "Parque Urbano", "Mercado Histórico", "Monumento Nacional", "Mirante Panorâmico"). Seja específico.
        - "descricao_enxuta": (string) Uma descrição concisa e informativa (1-2 frases) que justifique sua inclusão, destacando seus principais atrativos.
        - "avaliacao_geral": (string, opcional) Uma estimativa da avaliação geral se for amplamente conhecida (ex: "Excelente", "Muito Popular", "4.5/5 estrelas"). Use "N/A" se não houver uma estimativa clara.
        - "destaque_principal": (string) O motivo principal pelo qual um turista deveria visitar (ex: "Vistas incríveis da cidade", "Coleção de arte renascentista", "Arquitetura impressionante", "Atmosfera vibrante").

        Exemplo de um item na lista JSON:
        {{
          "nome": "Museu do Louvre",
          "tipo_principal": "Museu de Arte e Antiguidades",
          "descricao_enxuta": "Um dos maiores e mais visitados museus do mundo, lar de obras-primas como a Mona Lisa e a Vênus de Milo.",
          "avaliacao_geral": "4.7/5 estrelas",
          "destaque_principal": "Coleção de arte de renome mundial"
        }}

        Garanta que a saída seja SOMENTE a lista JSON, sem nenhum texto introdutório, comentários ou formatação adicional.
        """

        try:
            response = self.modelo.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.4, # Um pouco mais factual, menos aleatório
                    max_output_tokens=3072 # Espaço suficiente para a lista JSON
                ),
                safety_settings=self.safety_settings
            )

            json_text = response.text.strip()
            # Tentativa robusta de extrair JSON, mesmo que o modelo adicione ```json ... ```
            if json_text.startswith("```json"):
                json_text = json_text[7:]
            if json_text.endswith("```"):
                json_text = json_text[:-3]
            json_text = json_text.strip()

            if not json_text:
                display(Markdown(f"<font color='orange'>Aviso: Gemini retornou uma resposta vazia para {cidade}.</font>"))
                return []

            atracoes = json.loads(json_text)
            # Garantir que é uma lista, mesmo que o Gemini retorne algo diferente
            if not isinstance(atracoes, list):
                 display(Markdown(f"<font color='orange'>Aviso: Gemini retornou um formato inesperado (não uma lista) para {cidade}.</font>"))
                 return []

            display(Markdown(f"✅ *Gemini retornou {len(atracoes)} sugestões de atrações para {cidade}.*"))
            return atracoes[:top_n]
        except json.JSONDecodeError as e:
            display(Markdown(f"<font color='red'>**Erro (JSONDecodeError) ao processar resposta do Gemini para {cidade}:** {e}. "
                             "Isso geralmente ocorre se o modelo não retornar um JSON válido.</font>"))
            if hasattr(response, 'text'):
                display(Markdown(f"<pre>Resposta Bruta do Gemini:\n{response.text}</pre>"))
            else:
                 display(Markdown(f"<pre>Nenhuma resposta de texto recebida do Gemini.</pre>"))
            return []
        except Exception as e:
            # Captura de erros mais genéricos, como problemas de API (quota, etc.)
            # ou bloqueios de segurança não esperados.
            display(Markdown(f"<font color='red'>**Erro inesperado ao consultar Gemini para {cidade}:** {e}</font>"))
            if hasattr(response, 'prompt_feedback'):
                display(Markdown(f"Feedback do Prompt: {response.prompt_feedback}"))
            return []

    def pesquisar_destino(self, cidade: str, pais: str, top_n_sugestoes: int = 7) -> dict:
        """
        Executa a pesquisa de atrações para um destino específico.
        """
        display(Markdown(f"\n### 🌍 Agente Iniciando Pesquisa para: **{cidade}, {pais}**"))

        links = gerar_links_pesquisa_google(cidade, pais)
        sugestoes_gemini = self._obter_sugestoes_gemini(cidade, pais, top_n=top_n_sugestoes)

        display(Markdown(f"--- Pesquisa para **{cidade}, {pais}** finalizada pelo agente. ---"))
        return {
            "cidade": cidade,
            "pais": pais,
            "link_google_search": links[0],
            "link_google_maps_atracoes": links[1],
            "link_google_maps_cidade": links[2],
            "sugestoes_gemini": sugestoes_gemini
        }

# --- Função Principal de Execução ---
def executar_planejador_viagem(caminho_arquivo_csv: str = 'europa.csv'):
    """
    Orquestra o carregamento dos dados da viagem e a pesquisa de atrações para cada destino.
    Recebe o caminho do arquivo CSV como parâmetro.
    """
    if not GOOGLE_API_KEY or not gemini_model_instance:
        display(Markdown("<font color='red'>**Execução Interrompida.** API Key do Google ou modelo Gemini não configurados corretamente na inicialização.</font>"))
        return

    df_viagem = carregar_dados_viagem(caminho_arquivo_csv)

    if df_viagem is None:
        display(Markdown("<font color='red'>**Não foi possível prosseguir sem os dados da viagem.**</font>"))
        return

    agente_pesquisador = AgentePesquisadorAtracoes(modelo_llm=gemini_model_instance)

    dados_compilados = {
        "titulo_pesquisa": "Relatório de Pesquisa de Atrações do Agente",
        "data_execucao_pesquisa": time.strftime("%Y-%m-%d %H:%M:%S"),
        "data_geracao_relatorio_utc": time.strftime("%Y-%m-%d %H:%M:%S UTC"),
        "data_geracao_relatorio_brasilia": time.strftime("%Y-%m-%d %H:%M:%S BRT"),
        "destinos_pesquisados": []
    }


    display(Markdown("\n# 🗺️ **Relatório de Pesquisa de Atrações do Agente** ✈️"))

    resultados_completos = []
    for indice, linha_viagem in df_viagem.iterrows():
        cidade = linha_viagem['cidade']
        pais = linha_viagem['pais']

        resultado_destino = agente_pesquisador.pesquisar_destino(cidade, pais, top_n_sugestoes=15)

        # Adicionar dados da viagem original ao resultado
        resultado_destino['data_chegada'] = linha_viagem['data_chegada']
        resultado_destino['data_partida'] = linha_viagem['data_partida']
        resultado_destino['hospedagem'] = linha_viagem['hospedagem']
        resultados_completos.append(resultado_destino)

        # Exibição imediata por destino
        display(Markdown(f"\n## 📍 Destino: **{resultado_destino['cidade']}, {resultado_destino['pais']}**"))
        display(Markdown(f"🗓️ **Período:** {resultado_destino['data_chegada']} a {resultado_destino['data_partida']}"))
        display(Markdown(f"🏨 **Hospedagem:** {resultado_destino['hospedagem']}"))

        display(Markdown("#### 🔍 Links Úteis para Pesquisa Manual Detalhada (abrem em nova aba):"))
        display(Markdown(f"- **[Google Search: Principais atrações]({resultado_destino['link_google_search']})** ")) # Removido {} extra
        display(Markdown(f"- **[Google Maps: Atrações na área]({resultado_destino['link_google_maps_atracoes']})** ")) # Removido {} extra
        display(Markdown(f"- **[Google Maps: Visão geral da cidade]({resultado_destino['link_google_maps_cidade']})** ")) # Removido {} extra

        if resultado_destino['sugestoes_gemini']:
            display(Markdown("#### ✨ Sugestões de Atrações Principais (via Agente Gemini):"))

            tabela_atracoes_data = []
            # Chaves como definidas no prompt do Gemini
            headers = ["Nº", "Atração", "Tipo Principal", "Descrição Enxuta", "Avaliação Geral", "Destaque Principal"]

            for i, atracao in enumerate(resultado_destino['sugestoes_gemini']):
                tabela_atracoes_data.append([
                    i + 1,
                    atracao.get('nome', 'N/A'),
                    atracao.get('tipo_principal', 'N/A'),
                    atracao.get('descricao_enxuta', 'N/A'),
                    atracao.get('avaliacao_geral', 'N/A'),
                    atracao.get('destaque_principal', 'N/A')
                ])

            tabela_formatada = tabulate(tabela_atracoes_data, headers=headers, tablefmt="pipe", stralign="left")
            display(Markdown(tabela_formatada))
        else:
            display(Markdown("<font color='orange'>⚠️ Nenhuma sugestão de atração específica retornada pelo Gemini para este destino. "
                             "Utilize os links de pesquisa manual acima.</font>"))
        display(Markdown("---")) # Separador visual entre destinos

    # Você pode querer fazer algo com 'resultados_completos' aqui, como salvar em um JSON.
    # Exemplo:
    with open('planejamento_viagem_atracoes.json', 'w', encoding='utf-8') as f:
        json.dump(resultados_completos, f, ensure_ascii=False, indent=4)
    display(Markdown("\n💾 *Relatório completo também salvo em `planejamento_viagem_atracoes.json`.*"))
    return resultados_completos

# --- Ponto de Entrada da Execução ---
if __name__ == "__main__":
    # Certifique-se de que as bibliotecas estão instaladas
    # Em um notebook Colab, você executaria !pip install -q pandas google-generativeai tabulate em uma célula separada.
    # Agora você pode chamar a função passando o caminho do seu arquivo CSV, por exemplo:
    # executar_planejador_viagem('meu_arquivo_viagem.csv')
    # Se nenhum caminho for fornecido, ele usará 'europa.csv' por padrão.
    executar_planejador_viagem()

✅ *SDK do Gemini e modelo (`gemini-1.5-flash-latest`) configurados com sucesso.*

### Arquivo de Viagem (`europa.csv`)

,cidade,pais,data_chegada,data_partida,hospedagem
0,Roma,Itália,2025-09-13,2025-09-16,Via Francesco Caracciolo n.7
1,Florença,Itália,2025-09-16,2025-09-19,Hotel Bodoni
2,Viena,Aústria,2025-09-20,2025-09-24,Hilton Vienna Park



# 🗺️ **Relatório de Pesquisa de Atrações do Agente** ✈️


### 🌍 Agente Iniciando Pesquisa para: **Roma, Itália**

...

- **[Google Maps: Visão geral da cidade](https://www.google.com/maps/place/Viena%2C+A%C3%BAstria)** 

#### ✨ Sugestões de Atrações Principais (via Agente Gemini):

|   Nº | Atração                       | Tipo Principal                 | Descrição Enxuta                                                                                                         | Avaliação Geral   | Destaque Principal                             |
|-----:|:------------------------------|:-------------------------------|:-------------------------------------------------------------------------------------------------------------------------|:------------------|:-----------------------------------------------|
|    1 | Palácio de Schönbrunn         | Palácio Histórico e Jardins    | Residência de verão dos Habsburgos, com belos jardins, fontes e museus. Um exemplo magnífico da arquitetura barroca.     | Excelente         | Arquitetura imperial e vastos jardins          |
|    2 | Hofburg                       | Complexo de Palácios           | Antiga residência de inverno dos Habsburgos, abrigando museus, capelas e apartamentos imperiais.                         | Muito Popular     | História imperial e arquitetura diversificada  |
|    3 | Catedral de Santo Estêvão     | Catedral Gótica                | Símbolo de Viena, com sua arquitetura gótica impressionante e tesouros históricos.                                       | Excelente         | Arquitetura gótica e significado histórico     |
|    4 | Museu de História da Arte     | Museu de Arte                  | Coleção excepcional de pinturas, esculturas e objetos de arte de diferentes épocas e culturas.                           | 4.5/5 estrelas    | Coleção de arte abrangente e de alta qualidade |
|    5 | Museu Albertina               | Museu de Arte Gráfica          | Uma das maiores e mais importantes coleções de arte gráfica do mundo, com obras de Dürer a Picasso.                      | 4.5/5 estrelas    | Coleção excepcional de gravuras e desenhos     |
|    6 | Belvedere                     | Palácio e Museu de Arte        | Palácio barroco que abriga a maior coleção de obras de Gustav Klimt, incluindo 'O Beijo'.                                | Excelente         | Obras-primas de Gustav Klimt                   |
|    7 | Naschmarkt                    | Mercado Histórico              | Um dos maiores e mais vibrantes mercados ao ar livre da Europa, com uma grande variedade de produtos.                    | Muito Popular     | Experiência cultural e gastronômica vibrante   |
|    8 | Prater                        | Parque de Diversões            | Parque de diversões famoso pela icônica roda-gigante, oferecendo entretenimento para todas as idades.                    | Muito Popular     | Entretenimento e diversão para toda a família  |
|    9 | Roda Gigante do Prater        | Atração Turística              | Símbolo icônico de Viena, oferecendo vistas panorâmicas deslumbrantes da cidade.                                         | Excelente         | Vistas panorâmicas de Viena                    |
|   10 | Palácio do Belvedere Superior | Palácio Barroco e Museu        | Palácio barroco com jardins exuberantes, abrigando obras-primas da arte austríaca, incluindo o 'Beijo' de Klimt.         | Excelente         | Coleção de arte austríaca, incluindo Klimt     |
|   11 | Espaço de Arte Leopold        | Museu de Arte Moderna          | Museu dedicado à arte austríaca do século XX, com uma coleção impressionante de obras expressionistas e modernistas.     | 4.0/5 estrelas    | Arte austríaca moderna e expressionista        |
|   12 | Casa da Música                | Centro de Música e Arquitetura | Centro de música moderna com arquitetura inovadora e concertos regulares.                                                | 4.5/5 estrelas    | Arquitetura moderna e concertos                |
|   13 | Stephansplatz                 | Praça Histórica                | Praça central de Viena, localizada em frente à Catedral de Santo Estêvão, com edifícios históricos e atmosfera vibrante. | Muito Popular     | Coração histórico de Viena                     |
|   14 | Volksgarten                   | Parque Urbano                  | Belo parque no centro de Viena, com jardins, monumentos e o famoso Teatro de Verão.                                      | Muito Popular     | Espaço verde no centro da cidade               |
|   15 | Danúbio                       | Rio e Paisagem Urbana          | O rio Danúbio atravessa Viena, oferecendo oportunidades para passeios de barco e vistas deslumbrantes da cidade.         | N/A               | Vistas panorâmicas e atividades ao ar livre    |

---


💾 *Relatório completo também salvo em `planejamento_viagem_atracoes.json`.*

In [6]:
dados_compilados = executar_planejador_viagem("/content/europa.csv")

### Arquivo de Viagem (`/content/europa.csv`)

,cidade,pais,data_chegada,data_partida,hospedagem
0,Roma,Itália,2025-09-13,2025-09-16,Via Francesco Caracciolo n.7
1,Florença,Itália,2025-09-16,2025-09-19,Hotel Bodoni
2,Viena,Aústria,2025-09-20,2025-09-24,Hilton Vienna Park



# 🗺️ **Relatório de Pesquisa de Atrações do Agente** ✈️


### 🌍 Agente Iniciando Pesquisa para: **Roma, Itália**

🧠 *Agente consultando Gemini sobre atrações em: **Roma, Itália** (Top 15)...*

...

- **[Google Maps: Visão geral da cidade](https://www.google.com/maps/place/Viena%2C+A%C3%BAstria)** 

#### ✨ Sugestões de Atrações Principais (via Agente Gemini):

|   Nº | Atração                       | Tipo Principal                 | Descrição Enxuta                                                                                                                                           | Avaliação Geral   | Destaque Principal                                   |
|-----:|:------------------------------|:-------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------|:-----------------------------------------------------|
|    1 | Palácio de Hofburg            | Complexo de Palácios Imperiais | Residência de inverno dos Habsburgos por séculos, abriga museus, apartamentos imperiais e a Escola Espanhola de Equitação.                                 | Excelente         | Arquitetura imperial e história Habsburgo            |
|    2 | Palácio de Schönbrunn         | Palácio Barroco e Jardins      | Residência de verão dos Habsburgos, com magníficos jardins, fontes e o Gloriette, oferecendo vistas panorâmicas.                                           | Excelente         | Magnificência barroca e jardins extensos             |
|    3 | Catedral de Santo Estêvão     | Catedral Gótica                | Símbolo icônico de Viena, com sua arquitetura gótica impressionante, tesouros religiosos e a possibilidade de subir ao topo da torre sul.                  | Excelente         | Arquitetura gótica e vistas panorâmicas              |
|    4 | Museu de História da Arte     | Museu de Arte                  | Coleção excepcional de pinturas, esculturas e objetos de arte desde a antiguidade até o século XVIII, incluindo obras de Rembrandt, Rafael e Michelangelo. | Excelente         | Coleção de arte mundialmente famosa                  |
|    5 | Museu Albertina               | Museu de Arte Gráfica          | Uma das maiores e mais importantes coleções de arte gráfica do mundo, com obras de Dürer, Michelangelo e Picasso.                                          | Muito Popular     | Coleção abrangente de gravuras e desenhos            |
|    6 | Belvedere                     | Museu de Arte e Palácio        | Palácio barroco que abriga a maior coleção de obras de Gustav Klimt, incluindo 'O Beijo'.                                                                  | Excelente         | Obras-primas de Gustav Klimt                         |
|    7 | Naschmarkt                    | Mercado Histórico              | Um dos maiores e mais antigos mercados ao ar livre da Europa, oferecendo uma variedade de produtos frescos, especiarias e comidas internacionais.          | Muito Popular     | Experiência gastronômica e cultural vibrante         |
|    8 | Prater                        | Parque de Diversões            | Parque de diversões histórico e famoso pela icônica roda-gigante Wiener Riesenrad, oferecendo entretenimento para todas as idades.                         | Muito Popular     | Entretenimento clássico e vistas da cidade           |
|    9 | Roda-gigante Wiener Riesenrad | Atração de Parque de Diversões | Uma das mais antigas e famosas rodas-gigantes do mundo, localizada no Prater, oferecendo vistas panorâmicas de Viena.                                      | Excelente         | Vistas panorâmicas de Viena                          |
|   10 | Stephansplatz                 | Praça Pública                  | A principal praça de Viena, localizada em frente à Catedral de Santo Estêvão, com arquitetura histórica e atmosfera vibrante.                              | Muito Popular     | Coração histórico de Viena                           |
|   11 | Casa da Música                | Centro de Música e Arquitetura | Centro de música moderna com arquitetura inovadora e concertos de música clássica e contemporânea.                                                         | Muito Popular     | Arquitetura moderna e concertos de música            |
|   12 | Volksgarten                   | Parque Urbano                  | Parque público histórico e tranquilo, com jardins bem cuidados, monumentos e o famoso Templo de Teseu.                                                     | Muito Popular     | Espaço verde tranquilo no coração da cidade          |
|   13 | Danúbio                       | Rio e Passeios de Barco        | O rio Danúbio atravessa Viena, oferecendo passeios de barco cênicos e vistas da cidade a partir da água.                                                   | Muito Popular     | Passeios de barco e vistas da cidade a partir do rio |
|   14 | MuseumsQuartier               | Complexo de Museus             | Complexo de museus modernos e contemporâneos, incluindo o Leopold Museum e o MUMOK, com arquitetura impressionante e eventos culturais.                    | Muito Popular     | Diversidade de museus e arquitetura moderna          |
|   15 | Schlosspark Schönbrunn        | Jardim Histórico               | Os extensos jardins do Palácio de Schönbrunn, com fontes, esculturas, e o labirinto, oferecem um passeio tranquilo e paisagístico.                         | Excelente         | Paisagismo barroco e atmosfera serena                |

---


💾 *Relatório completo também salvo em `planejamento_viagem_atracoes.json`.*

In [7]:
from datetime import datetime, timedelta, timezone

# --- INÍCIO: Adapte esta seção com os dados reais da sua pesquisa ---
# Suponha que esta variável (ou um conjunto de variáveis) contenha
# todas as informações compiladas pelo seu Agente "Pesquisador de Atrações".
# O exemplo abaixo mostra uma ESTRUTURA SUGERIDA para organizar os dados
# para facilitar a geração do Markdown. Você precisará preenchê-la ou
# adaptar o código de geração do Markdown à sua estrutura de dados existente.

# Obtenção da data e hora atuais para o relatório
# Em um ambiente Colab real, para obter a hora dinâmica, você usaria:
# current_time_utc = datetime.now(timezone.utc)


# --- FIM: Adapte esta seção com os dados reais da sua pesquisa ---

# Construção do conteúdo em formato Markdown
markdown_output = []

markdown_output.append(f"# {dados_compilados.get('titulo_pesquisa', 'Resultados da Pesquisa de Atrações')}")
markdown_output.append(f"\n**Data da Pesquisa (Execução do Agente):** {dados_compilados.get('data_execucao_pesquisa', 'Não especificada')}")
markdown_output.append(f"**Data de Geração deste Relatório:** {dados_compilados.get('data_geracao_relatorio_utc')} / {dados_compilados.get('data_geracao_relatorio_brasilia')}\n")

markdown_output.append(f"## Introdução\n{dados_compilados.get('introducao', 'Introdução não fornecida.')}\n")

if dados_compilados.get('atracoes_pesquisadas'):
    markdown_output.append("## Detalhamento das Atrações Pesquisadas")
    for atracao in dados_compilados['atracoes_pesquisadas']:
        markdown_output.append(f"\n### {atracao.get('nome_atracao', 'Atração Sem Nome')}")

        if atracao.get('dados_brutos'):
            markdown_output.append("\n#### Dados Brutos / Informações Essenciais")
            for chave, valor in atracao['dados_brutos'].items():
                if isinstance(valor, str) and valor.startswith("http"):
                    markdown_output.append(f"- **{chave}:** [{valor}]({valor})")
                else:
                    markdown_output.append(f"- **{chave}:** {valor}")

        markdown_output.append(f"\n#### Análise Detalhada\n{atracao.get('analise_detalhada', 'N/A')}")
        markdown_output.append(f"\n#### Conclusões e Recomendações\n{atracao.get('conclusoes_recomendacoes', 'N/A')}")

        if atracao.get('fontes_consultadas'):
            markdown_output.append("\n##### Fontes Consultadas:")
            for fonte in atracao['fontes_consultadas']:
                markdown_output.append(f"- [{fonte}]({fonte})") # Formata como link clicável

        markdown_output.append(f"\n##### Insights Adicionais\n{atracao.get('insights_adicionais', 'Nenhum insight adicional.')}\n")

markdown_output.append(f"## Resumo Executivo da Pesquisa\n{dados_compilados.get('resumo_executivo_pesquisa', 'Resumo não fornecido.')}\n")
markdown_output.append(f"## Considerações para o Agente Organizador de Roteiro\n{dados_compilados.get('consideracoes_para_organizador_roteiro', 'Nenhuma consideração específica.')}\n")

conteudo_final_markdown = "\n".join(markdown_output)

# Especificações do arquivo de saída
nome_arquivo = "pesquisa_agente.md"
localizacao_arquivo = f"/content/{nome_arquivo}" # Diretamente na raiz do /content/

# Comando para salvar o arquivo
try:
    with open(localizacao_arquivo, "w", encoding="utf-8") as f:
        f.write(conteudo_final_markdown)
    print(f"Arquivo '{localizacao_arquivo}' gerado com sucesso! 🎉")
except Exception as e:
    print(f"Ocorreu um erro ao tentar salvar o arquivo '{localizacao_arquivo}': {e} 😥")

# Salva todos os dados de atrações em um arquivo JSON
output_filename = "atracoes_pesquisadas.json"
with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(agente.all_attractions_data, f, ensure_ascii=False, indent=4)

print(f"Dados salvos em {output_filename}")


AttributeError: 'list' object has no attribute 'get'